In [1]:
import numpy as np
from collections import defaultdict as dd

In [2]:

file = open('supertags.txt', 'r')

tags = dict()
for i in file:
    e, f = i.split(' ')
    if f[-1] == '\n':
        f = f[:-1]
    tags[e] = f
# i = 0
# for e in tags:
#     print(e, tags[e])
#     i += 1
#     if i > 10:
#         break

def tag(s):
    if s in tags.keys():
        return tags[s]
    else:
        return ('^' + s)[-3:]

file.close()

In [3]:
file = open('unigrams.txt', 'r')
unigrams = dict()
for e in file:
    if e[-1] == '\n':
        e = e[:-1]
    e = e.lower()
    first, second = e.split()
    unigrams[first] = int(second)

file.close()

# Zadanie 3

In [4]:
korp = dd(list)
for e in unigrams:
    tg = tag(e)
    korp[tg].append((e, unigrams[e]))
    
# i = 0
# for e in korp:
#     print(e, korp[e])
#     i += 1
#     if i > 10:
#         break

In [5]:
def generate_sentence_unigram(sent):
    sent = [x for x in map(lambda x: tag(x), sent.lower().split(' '))]
    res = []
    for e in sent:
        words = np.array([x[0] for x in korp[e]])
        probs = np.array([x[1] for x in korp[e]])
        probs = probs/np.sum(probs)
        res.append(np.random.choice(words, size=1, p=probs)[0])
    return ' '.join(res)

In [6]:
generate_sentence_unigram('Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę')
generate_sentence_unigram('W zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe')


'w ogłoszeniu jednym powinieneś grać dzieciństwa o uwarunkowaniach ze pewną kontynuacją antypartyjną jak górnictwo publiczne'

In [7]:
bigrams = dd(list)
file = open('2grams.txt', 'r')
# i = 0
for e in file:
    if e[-1] == '\n':
        e = e[:-1]
    l, first, second = e.split(' ')
    bigrams[first].append((int(l), second))
#     i += 1
    if int(l) <= 1:
#         print(l)
        break

file.close()

# Zadanie 4

In [8]:
korp_b = dd(list)
for e in bigrams:
    for f, g in bigrams[e]:
        tag2 = tag(g)
        korp_b[(e, tag2)].append((g, f))

# i = 0
# for e in korp_b:
#     print(e, korp_b[e])
#     i += 1
#     if i > 10:
#         break

In [9]:
def backtracking_unigram(tag_list):
    tag = tag_list[0]
#     print(korp[tag][0][0])
    res = generate_sentence_unigram(korp[tag][0][0])
#     backtracks = 0
    if len(tag_list) <= 1:
        return ['|', res]
    while True:
        nexts = backtracking(res, tag_list[1:])
        if nexts:
            return ['|', res] + nexts
        res = generate_sentence_unigram(korp[tag][0][0])

def backtracking(s, tag_list):
    tag = tag_list[0]
    if (s, tag) not in korp_b:
        return False
    backtracks = 0
    words = np.array([x[0] for x in korp_b[(s, tag)]])
    probs = np.array([x[1] for x in korp_b[(s, tag)]])
    probs = probs/np.sum(probs)
    res = np.random.choice(words, size=1, p=probs)
    if len(tag_list) <= 1:
        return [res[0]]
    while True:
#         print(backtracks)
        nexts = backtracking(res[0], tag_list[1:]) 
        if not nexts:
            backtracks += 1
        else:
#             print(res[0], nexts)
            return [res[0]] + nexts
        if backtracks > 50:
            return backtracking_unigram(tag_list)
        res = np.random.choice(words, size=1, p=probs)

def generate_sentence_bigram(sent):
    sent = [x for x in map(lambda x: tag(x), sent.lower().split(' '))]
    res = backtracking_unigram(sent)
    return ' '.join(res)

In [10]:
print(generate_sentence_bigram('Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę'))
print(generate_sentence_bigram('przed ćwiczeniami studenci dostarczają prowadzącemu deklaracje'))
print(generate_sentence_bigram('W zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe'))

| stary profesor potwierdził w niedzielnej porażce ponownie wystawioną fakturę
| na zadaniami | dziadkowie | dają zamawiającemu informacje
| w obliczu tym powinieneś pamiętać słowa o świadczeniach z podwójną mocą prawną jak ministerstwo wojskowe
